In [ ]:
import os
import pickle
import torch
import torchvision
import numpy as np
import pandas as pd
import torch.nn as nn
from PIL import Image
from tqdm import tqdm
import torch.nn as nn
import torch.utils.data as data1
from torch.utils import data
import torch.nn.functional as F
import torchvision.models as models
import matplotlib.pyplot as plt
from torch.autograd import Variable
from sklearn.metrics import accuracy_score
import torchvision.transforms as transforms
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorboardX import SummaryWriter
import torch.nn as nn
import torch

In [ ]:
class C3D(nn.Module):
    def __init__(self, img_dim, frames,dropout):
        super(C3D, self).__init__()

        self.conv1 = nn.Conv3d(3, 32, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.pool1 = nn.MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2))
        self.bn1 = nn.BatchNorm3d(32)

        self.conv2 = nn.Conv3d(32, 64, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.pool2 = nn.MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2))
        self.bn2 = nn.BatchNorm3d(64)
        
        self.conv3a = nn.Conv3d(64, 64, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.conv3b = nn.Conv3d(64, 64, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.pool3 = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2))
        self.bn3 = nn.BatchNorm3d(64)
        
        self.conv4a = nn.Conv3d(64, 128, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.conv4b = nn.Conv3d(128, 128, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.pool4 = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2))
        self.bn4 = nn.BatchNorm3d(128)
        
        self.conv5a = nn.Conv3d(128, 256, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.conv5b = nn.Conv3d(256, 256, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.pool5 = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=(0, 1, 1))
        
        dim_shape = new_output_shape(num_of_maxpool_2=5, shape=img_dim)
        channel_shape = new_channel_shape(frames=frames) # As first time channel is not modified
        self.fc_conv = nn.Conv3d(256, 2, kernel_size=(channel_shape, dim_shape, dim_shape))
        
        self.dropout = nn.Dropout(p=dropout)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = x.type(torch.cuda.FloatTensor)
        x = x.permute(0,4,1,2,3)

        h = self.relu(self.conv1(x))
        h = self.bn1(h)
        h = self.pool1(h)
        h = self.dropout(h)
        
        h = self.relu(self.conv2(h))
        h = self.bn2(h)
        h = self.pool2(h)
        h = self.dropout(h)

        h = self.relu(self.conv3a(h))
        h = self.relu(self.conv3b(h))
        h = self.bn3(h)
        h = self.pool3(h)
        h = self.dropout(h)
        
        h = self.relu(self.conv4a(h))
        h = self.relu(self.conv4b(h))
        h = self.bn4(h)
        h = self.pool4(h)
        h = self.dropout(h)
        
        h = self.relu(self.conv5a(h))
        h = self.relu(self.conv5b(h))
        h = self.pool5(h)
        h = self.fc_conv(h)
        
        logits = h.view(x.shape[0],-1)
        return logits

resize and crop image, and stack them together alongg with label y=0 or 1 if orignal or fake

In [ ]:
from torch.utils import data
class Dataload_3D_CNN(data.Dataset):
    def __init__(self, data_path, transform=None):
        self.transform = transform
        self.folders = data_path

    def read_images(self, data_path, use_transform):
        X = []
        for i in os.listdir(data_path):
            image = Image.open(os.path.join(data_path,i))
            
            if use_transform is not None:
                image = use_transform(image)
            image = torch.from_numpy(np.asarray(image))
            X.append(image)
        X = torch.stack(X, dim=0)
        return X

    def __getitem__(self, index):
        data_path = os.path.join(self.folders,os.listdir(self.folders)[index])
        
        X = self.read_images(data_path, self.transform)
        y = 1
        if 'orig' in data_path:
            y = 0
        return X, torch.from_numpy(np.array(y)).type(torch.LongTensor)

In [ ]:
TRANSFORM_IMG = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(256),
    ])

In [ ]:
train_path = './data/train/'
train_data = Dataload_3D_CNN(train_path, transform=TRANSFORM_IMG)
val_path = './data/val/'
val_data = Dataload_3D_CNN(val_path, transform=TRANSFORM_IMG)

cnn3d = C3D(img_dim=256, frames=10, dropout=0.4)
optimizer = torch.optim.Adam(cnn3d.parameters(), lr=learning_rate)

In [ ]:
epochs = 40
batch_size = 8
learning_rate = 1e-4
log_interval = 10
img_x, img_y = 256,256

loss_fn = nn.CrossEntropyLoss()
fc_hidden1, fc_hidden2 = 256, 256
dropout = 0.0
begin_frame, end_frame, skip_frame = 1, 10, 1

In [ ]:
def train(log_interval, model, device, train_loader, optimizer, epoch):
    model.train()

    losses = []
    scores = []
    N_count = 0
    for batch_idx, (X, y) in enumerate(train_loader):
        N_count += X.size(0)
        optimizer.zero_grad()
        output = model(X)
        loss = F.cross_entropy(output, y)
        losses.append(loss.item())

        y_pred = torch.max(output, 1)[1]
        step_score = accuracy_score(y.data.squeeze().numpy(), y_pred.data.squeeze().numpy())
        scores.append(step_score)

        loss.backward()
        optimizer.step()
        
    return np.mean(losses), np.mean(scores)

In [ ]:
for epoch in range(epochs):
    train_losses, train_scores = train(log_interval, cnn3d, device, train_loader, optimizer, epoch)
    print("Training Loss : "+str(train_losses))